In [ ]:
!pip install -r requirements.txt
%load_ext autoreload
%autoreload 2

In [ ]:
import girder
import json
import os
import pandas as pd
import postgres_to_girder
import psycopg2
import requests
with open ("config.json", "r") as j:
    config = json.load(j)
with open ("context.json", "r") as j:
    context = json.load(j)

In [ ]:
postgres_to_girder.girder_login(
    config["girder"]["user"],
    config["girder"]["password"],
    config["girder"]["host"]
)

In [ ]:
activities_id = postgres_to_girder.get_girder_id_by_name(
    entity="collection",
    name="activities",
    host=config["girder"]["host"]
)

In [ ]:
activities_id

In [ ]:
try:
    conn = psycopg2.connect(
        " ".join(
            [
                "=".join([
                    key,
                    config["postgres"][key]
                ]) for key in config["postgres"]
            ]
        )
    )
except:
    print("I am unable to connect to the database")

In [ ]:
acts = pd.io.sql.read_sql_query(
    "SELECT * FROM acts;",
    conn
)
users = pd.io.sql.read_sql_query(
    "SELECT * FROM users;",
    conn
)

In [ ]:
for activity in set(acts["title"]):
    if not postgres_to_girder.get_girder_id_by_name(
        "folder",
        activity,
        parent=(
            "collection",
            activities_id
        ),
        host=config["girder"]["host"]
    ):
        postgres_to_girder.girder_login(
            config["girder"]["user"],
            config["girder"]["password"],
            config["girder"]["host"]
        )
        print(
            postgres_to_girder.create_girder_entity(
                entity="folder",
                name=activity,
                host=config["girder"]["host"],
                parent=(
                    "collection",
                    activities_id
                )
            )
        )